# Socket Server

In [8]:
import socketserver
import socket
import sys 
import threading
import time
import hmac
import os

In [9]:
def auth(conn):
    msg = os.urandom(32)    # 生成一个随机的字符串
    print(msg, len(msg))    # 32位
    conn.send(msg)  # 发送到client端
    result = hmac.new(secret_key, msg)  # 处理这个随机字符串，得到一个结果
    client_digest = conn.recv(1024)     # 接收client端处理的结果
    if result.hexdigest() == client_digest.decode('utf-8'):
        print('是合法的连接')     # 对比成功可以继续通讯
        return True
    else:
        print('不合法的连接')     # 不成功 close
        return False

In [ ]:
def socket_server():
    try:
        secret_key = b'alex_s'   # 相当于'盐'
        # 首先调用socket函数创建一个socket
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # 防止socket server重启端口被占用（socket.error:[Errno 98] Address already in use）
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # 调用bind函数将其与本机地址以及一个端口号绑定
        s.bind(('127.0.0.1', 6666))
        # listen在相应的socket上监听
        s.listen(10)
    except socket.error as msg:
        print(msg)
        sys.exit(1)
    print("waiting connection...")
    
    while 1:
        conn, addr = s.accept()
        if auth(conn):
            t = threading.Thread(target=deal_data, args=(conn, addr))
            t.start()
def deal_data(conn, addr):
    print("Accept new connection from {0}".format(addr))
    conn.send(("Welcome to the server.").encode())
    while 1:
        data = conn.recv(1024)
        print('{0} client send data is {1}'.format(addr, data.decode()))
        time.sleep(1)
        if data == 'exit' or not data:
            print('{0} connection close.'.format(addr))
            conn.send(bytes('Connetion closed!'), 'UTF-8')
            break
        conn.send(bytes('Hello, {0}'.format(data), 'UTF-8'))
    conn.close()
    
if __name__ == '__main__':
    socket_server()

waiting connection...


## 基于UDP

In [ ]:
from socket import *
serverPort=12345
serverSocket=socket(AF_INET,SOCK_DGRAM)
serverSocket.bind(('127.0.0.1',serverPort))
print('ready')
while 1:
    message,clientAddress=serverSocket.recvfrom(2048)
    modifiedMessage=message.upper()
    print(modifiedMessage)
    serverSocket.sendto(modifiedMessage,clientAddress)

ready
b'HI'


## 验证合法性-Server

In [5]:
import os
import hmac
import socket
def auth(conn):
    msg = os.urandom(32)    # 生成一个随机的字符串
    print(msg, len(msg))    # 32位
    conn.send(msg)  # 发送到client端
    result = hmac.new(secret_key, msg)  # 处理这个随机字符串，得到一个结果
    client_digest = conn.recv(1024)     # 接收client端处理的结果
    if result.hexdigest() == client_digest.decode('utf-8'):
        print('是合法的连接')     # 对比成功可以继续通讯
        return True
    else:
        print('不合法的连接')     # 不成功 close
        return False
secret_key = b'alex_s'   # 相当于'盐'

sk = socket.socket()

sk.bind(('127.0.0.1', 9000))

sk.listen()

conn, addr = sk.accept()

if auth(conn):

    print(conn.recv(1024))

    # 正常的和client端进行沟通

    conn.close()

else:

    conn.close()

sk.close()

b'0Dbq\xba\xcb\xbaTTA\x82\x00\x16\xbc\x15_xf\xc6UbJ=E\x97do\xd8D\xe1Y\x8a' 32


TypeError: Missing required parameter 'digestmod'.